# 감성분석

## 아마존 핸드폰 리뷰 데이터의 감성 분석

데이터 다운로드

In [3]:
import requests
res = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip')
with open('sentiment labelled sentences.zip', 'wb') as f:
    f.write(res.content)

압축 풀기

In [4]:
from zipfile import ZipFile
z = ZipFile('sentiment labelled sentences.zip')
data = z.open('sentiment labelled sentences/amazon_cells_labelled.txt')

파일 열기

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(data, sep="\t", header=None)

In [7]:
df.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [8]:
df.shape

(1000, 2)

단어 문서 행렬 만들기

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
cv = CountVectorizer(max_features=1000, stop_words='english')
# stop_words 불용어 한글의 경우 조사를 의미함. 

In [11]:
tdm = cv.fit_transform(df[0])

In [12]:
tdm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [13]:
cv.get_feature_names()

['10',
 '100',
 '11',
 '12',
 '13',
 '15',
 '15g',
 '18',
 '20',
 '2000',
 '2005',
 '2160',
 '24',
 '2mp',
 '325',
 '350',
 '375',
 '3o',
 '42',
 '44',
 '45',
 '4s',
 '50',
 '5020',
 '510',
 '5320',
 '680',
 '700w',
 '8125',
 '8525',
 '8530',
 'abhor',
 'ability',
 'able',
 'abound',
 'absolutel',
 'absolutely',
 'ac',
 'accept',
 'acceptable',
 'access',
 'accessory',
 'accessoryone',
 'accidentally',
 'accompanied',
 'actually',
 'ad',
 'adapter',
 'adapters',
 'add',
 'addition',
 'additional',
 'address',
 'adhesive',
 'adorable',
 'advertised',
 'advise',
 'aggravating',
 'ago',
 'alarm',
 'allot',
 'allow',
 'allowing',
 'allows',
 'alot',
 'aluminum',
 'amazed',
 'amazing',
 'amazon',
 'amp',
 'ample',
 'angeles',
 'angle',
 'answer',
 'ant',
 'antena',
 'anti',
 'apart',
 'apartment',
 'apparently',
 'appealing',
 'appearance',
 'appears',
 'applifies',
 'appointments',
 'area',
 'arguing',
 'armband',
 'arrival',
 'arrived',
 'asia',
 'ask',
 'aspect',
 'assumed',
 'atleast',


훈련용 데이터(800개)와 테스트용 데이터(200개) 나누기

In [14]:
x_train = tdm[:800]
x_test = tdm[800:]
y_train = df.iloc[:800, 1]
y_test = df.iloc[800:, 1]

## 선형 모형

In [1]:
import tensorflow as tf

C:\Users\hyoun\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

1000개의 단어를 입력 받아 1개의 예측(긍/부정)을 하는 선형 모형. 시그모이드(로지스틱) 활성화 함수를 사용.

In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(1000,), activation='sigmoid')) 

Instructions for updating:
Colocations handled automatically by placer.


모형 요약: 파라미터의 수는 단어별 가중치 1000개 + 절편 1개 = 1001개

In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 1001      
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


학습 준비. 학습 알고리즘(`optimizer`)는 `adam`을 사용. 손실함수(`loss`)는 교차 엔트로피(`binary_crossentropy`). 보조적인 지표로 정확도(accurary)를 사용. accuracy란 전체 사례 중 몇 맞은 사례의 비율

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

학습. `batch_size`는 한 번에 데이터를 입력하는 크기. 학습용 데이터가 모두 800개이므로 40개씩 20번 입력하면 모든 데이터를 한 번씩 입력하게 됨. 이를 1 에포크(epoch)라고 함. 아래는 총 10 에포크를 진행.

학습을 시키면 최종 loss는 0.6092, 정확도는 86%임. (실행할 때마다 달라질 수 있음)

In [18]:
model.fit(x_train, y_train, batch_size=40, epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
800/800 [==============================] - 1s 2ms/sample - loss: 0.6870 - acc: 0.5562
Epoch 2/10
800/800 [==============================] - 0s 101us/sample - loss: 0.6782 - acc: 0.6162
Epoch 3/10
800/800 [==============================] - 0s 105us/sample - loss: 0.6699 - acc: 0.6575
Epoch 4/10
800/800 [==============================] - 0s 100us/sample - loss: 0.6618 - acc: 0.7050
Epoch 5/10
800/800 [==============================] - 0s 101us/sample - loss: 0.6539 - acc: 0.7400
Epoch 6/10
800/800 [==============================] - 0s 98us/sample - loss: 0.6461 - acc: 0.7725
Epoch 7/10
800/800 [==============================] - 0s 99us/sample - loss: 0.6385 - acc: 0.8100
Epoch 8/10
800/800 [==============================] - 0s 98us/sample - loss: 0.6312 - acc: 0.8400
Epoch 9/10
800/800 [==============================] - 0s 95us/sample - loss: 0.6239 - acc: 0.8562
Epoch 10/10
800/800 [==============================] - 0s 98us/samp

테스트 데이터로 모형을 평가. loss는 0.6491, 정확도는 74%로 훈련용 데이터보다 성능이 낮아졌음. 과적합이 발생했음을 알 수 있음.

In [19]:
model.evaluate(x_test, y_test)

200/200 [==============================] - 0s 401us/sample - loss: 0.6578 - acc: 0.7200


[0.6577629709243774, 0.72]

## 가중치 확인

In [20]:
weight, bias = model.trainable_weights

In [21]:
word_weight = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '가중치': weight.numpy().flat
})

C:\Users\hyoun\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


가중치 순으로 정렬하여 가중치가 낮은(-)인 단어들을 확인. 이 단어들이 많이 나타나는 리뷰는 부정(0)일 확률이 높음

In [22]:
word_weight.sort_values('가중치').head()

,단어,가중치
988,worst,-0.180307
358,junk,-0.161844
803,terrible,-0.149051
713,signal,-0.146441
203,dead,-0.144985


가중치 순으로 정렬하여 가중치가 높은(+)인 단어들을 확인. 이 단어들이 많이 나타나는 리뷰는 긍정(1)일 확률이 높음

In [23]:
word_weight.sort_values('가중치').tail()

,단어,가중치
284,fits,0.150419
100,awesome,0.157434
114,best,0.186057
985,works,0.208661
310,great,0.235184


## 한국어 감성 분석

Naver Sentiment Movie Corpus를 이용해 한국어 감성 분석을 실시한다.

먼저 데이터를 다운받는다.

In [24]:
res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('nsmc_train.csv', 'wb') as f:
    f.write(res.content)

데이터를 연다.

In [25]:
nsmc = pd.read_csv('nsmc_train.csv', sep='\t')

In [26]:
nsmc.shape

(150000, 3)

In [27]:
nsmc.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


데이터가 많으므로 2000개만으로 실습을 진행한다.

In [28]:
nsmc = nsmc.loc[:1999]

인터넷 글의 경우 맞춤법이나 띄어쓰기를 잘 지키지 않으므로 형태소 분석을 하기보다는 `sentencepiece`를 이용해서 자주 나오는 글자 조합으로 준단어 토큰화한다. 속도 면에서도 형태소 분석보다는 준단어 토큰화가 빠르다.

`sentencepiece` 패키지가 설치되어 있지 않으면 먼저 설치해준다.

In [29]:
!pip install sentencepiece

리뷰를 `nsmc.txt` 파일로 저장한다.

In [30]:
with open('nsmc.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(nsmc['document']))

자주 나오는 글자 조합을 최대 2000개까지 찾는다.

In [31]:
from sentencepiece import SentencePieceTrainer
SentencePieceTrainer.Train('--input=nsmc.txt --model_prefix=nsmc --vocab_size=2000')

True

찾은 결과를 불러온다.

In [32]:
from sentencepiece import SentencePieceProcessor
sp = SentencePieceProcessor()
sp.Load("nsmc.model")

True

첫 번째 리뷰를 준단어 토큰화 해본다. `▁`는 띄어쓰기 된 부분을 나타낸다.

In [33]:
sp.encode_as_pieces(nsmc.loc[0, 'document'])

['▁아', '▁더빙', '..', '▁진짜', '▁짜증나', '네요', '▁목소리']

TDM을 만든다. `lowercase=False`로 설정하면 소문자로 변환을 하지 않는다.

In [34]:
cv = CountVectorizer(max_features=2000, lowercase=False, tokenizer=sp.encode_as_pieces)

In [35]:
tdm = cv.fit_transform(nsmc['document'])

학습용 데이터와 테스트용 데이터를 나눈다.

In [36]:
x_train = tdm[:1600]
x_test = tdm[1600:]
y_train = nsmc['label'][:1600]
y_test = nsmc['label'][1600:]

모형을 만들고 학습시킨다. 이 과정은 위와 같다.

In [37]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_shape=(2000,), activation='sigmoid'))

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [39]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1600/1600 [==============================] - 0s 267us/sample - loss: 0.6905 - acc: 0.5288
Epoch 2/10
1600/1600 [==============================] - 0s 136us/sample - loss: 0.6602 - acc: 0.6712
Epoch 3/10
1600/1600 [==============================] - 0s 123us/sample - loss: 0.6347 - acc: 0.7606
Epoch 4/10
1600/1600 [==============================] - 0s 123us/sample - loss: 0.6117 - acc: 0.8056
Epoch 5/10
1600/1600 [==============================] - 0s 121us/sample - loss: 0.5909 - acc: 0.8356
Epoch 6/10
1600/1600 [==============================] - 0s 119us/sample - loss: 0.5720 - acc: 0.8575
Epoch 7/10
1600/1600 [==============================] - 0s 148us/sample - loss: 0.5545 - acc: 0.8725
Epoch 8/10
1600/1600 [==============================] - 0s 148us/sample - loss: 0.5386 - acc: 0.8813
Epoch 9/10
1600/1600 [==============================] - 0s 145us/sample - loss: 0.5241 - acc: 0.8800
Epoch 10/10
1600/1600 [==============================] - 0s 136us/sample - loss: 0.5101 - a

In [40]:
model.evaluate(x_test, y_test)

400/400 [==============================] - 0s 293us/sample - loss: 0.5721 - acc: 0.7500


[0.5721164274215699, 0.75]

In [41]:
weight, bias = model.trainable_weights

In [42]:
word_weight = pd.DataFrame({
    '단어': cv.get_feature_names(),
    '가중치': weight.numpy().flat
})

C:\Users\hyoun\Anaconda3\lib\site-packages\pandas\core\dtypes\dtypes.py:458: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if string == 'category':


In [43]:
word_weight.sort_values('가중치').head()

,단어,가중치
525,▁재미없,-0.277594
248,▁돈,-0.243137
329,▁보지마,-0.229554
1608,전혀,-0.226512
593,▁최악의,-0.225748


In [44]:
word_weight.sort_values('가중치').tail()

,단어,가중치
287,▁명작,0.232164
535,▁재밌었,0.237064
1434,어요,0.243410
154,▁ᄒᄒ,0.261225
592,▁최고의,0.306250
